In [1]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup
import string

In [2]:
page = wptools.page('1983_United_States_embassy_bombing_in_Beirut')
page.get_parse()

en.wikipedia.org (parse) 1983_United_States_embassy_bombing_in_Beirut
en.wikipedia.org (imageinfo) File:Beirutembassy.jpg
1983 United States embassy bombing in Beirut (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Beirutemb...
  infobox: <dict(10)> title, image, caption, date, time, partof, t...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:1...
  pageid: 10533380
  parsetree: <str(19514)> <root><template><title>Short description...
  requests: <list(2)> parse, imageinfo
  title: 1983 United States embassy bombing in Beirut
  wikibase: Q548057
  wikidata_url: https://www.wikidata.org/wiki/Q548057
  wikitext: <str(16494)> {{Short description|1983 suicide bombing ...
}


In [3]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "1983_United_States_embassy_bombing_in_Beirut",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('1983 United States embassy bombing in Beirut')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
1978 South Lebanon conflict
1982 Beirut bombing
1982 Iranian diplomats kidnapping
1982 Lebanon War
1983 Beirut barracks bombing
1983 Beirut barracks bombings
1983 Kuwait bombings
1983 United States embassy bombing
1984 Sohmor massacre
1984 United States embassy annex bombing
1984 United States embassy annex bombing in Beirut
1985 Beirut car bombings
2013 Iranian embassy bombing
Ali Akbar Velayati
Amine Gemayel
Assassination of Bachir Gemayel
Assassination of René Moawad
At Tiri incident
Barry Goldwater
Battle of Sidon (1991)
Battle of Tripoli (1983)
Battle of Zahleh
Battle of the Hotels
Beirut
Black Saturday (Lebanon)
Bombing of Lebanon (June 1981)
Bureau of Diplomatic Security
Bus massacre
CIA
Car bomb
Central Intelligence Agency
Charles H. Percy
Claiborne Pell
Crusades
Dahr al-Wahsh
Damour massacre
Diplomatic Security Service
Ehden massacre
February 6 Intifada
GMT
Geographic coordinate system
Hezbollah
History of Hezbollah
House Foreign Affairs Committee
Hundred Days' War
Impe

In [4]:
def get_arabic_name(english_phrase):
    # if we remove all numbers from a string and the string remains empty - then we won't translate that
    english_phrase_mod = ''.join([i for i in english_phrase if not i.isdigit()])
    english_phrase_mod = english_phrase_mod.translate(str.maketrans('', '', string.punctuation))
    english_phrase_mod = english_phrase_mod.strip()
    if english_phrase_mod == '':
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1
    else:
        english_phrase = english_phrase_mod
        
    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
    s = re.sub('<Br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries
    
    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s


def jaccard(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [5]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [6]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [7]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1
                    
                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    print(name)
                    if isinstance(name, list):
                        if len(name) > 1:
                            print(name)
                            jsims = []
                            for i, n in enumerate(name):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name = [name[jsims.index(max(jsims))]]
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'
                    
                    df_political_party_en = df_political_party_en.append({
                        'Name': ','.join(name if isinstance(name, list) else name),
                        'Ideology': ','.join(ideology) if isinstance(ideology, list) else ideology,
                        'Political Position': ','.join(position) if isinstance(position, list) else position,
                        'Founder(s)': founders,
                    }, ignore_index=True)
                    
                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)
                    
                    
                    
                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']
                    
                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'
                    
                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')
                    
                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'
                    
                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')
                    
                    
                    df_political_party_ar = df_political_party_ar.append({
                        'الاسم': ','.join(name_ar),
                        'الإيديولوجيا': ','.join(ideology_ar),
                        'الموقف السياسي': ','.join(position_ar),
                        'المؤسس (المؤسسون)': ','.join(founders_ar),
                    }, ignore_index=True)
                    
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:PikiWiki Israel 4220 Israel Def...
en.wikipedia.org (parse) 1982 Beirut bombing
en.wikipedia.org (parse) 1982 Iranian diplomats kidnapping
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1983 Beirut barracks bombing
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 Beirut barracks bombings
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 Kuwait bombings
en.wikipedia.org (parse) 1983 United States embassy bombing
en.wikipedia.org (imageinfo) File:Beirutembassy.jpg
en.wikipedia.org (parse) 1984 Sohmor massacre
en.wikipedia.org (parse) 1984 United States embassy annex bombing
en.wikipedia.org (imageinfo) File:US Embassy in Beirut, Lebanon (...
en.wikipedia.org (parse) 1984 United States embassy annex bombing...
en.wikipedia.org (imageinfo) File:US Embassy in Beirut, Lebanon (...
en.wikipedia.org (parse) 1985 Beirut car bombing

[[Hassan Nasrallah|Sayyed Hassan Nasrallah]]
['Hezbollah']
-----------------
name:  ['Hezbollah']
founders:  Sayyed Hassan Nasrallah,
ideology:  ['Antisemitism', 'Khomeinism', 'Anti-West', 'Anti-imperialism', 'Unbulleted list', 'Pan-Islamism', 'Anti-Zionism', 'Shia Jihad', 'Islamic nationalism']
position:  -
START
name:  Hezbollah
name ar:  حزب الله 
name ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%84%D9%87
END
START
ideology:  Antisemitism
ideology ar:  معاداة السامية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%B3%D8%A7%D9%85%D9%8A%D8%A9
ideology:  Khomeinism
ideology ar:  خمينية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%AE%D9%85%D9%8A%D9%86%D9%8A%D8%A9
Cannot find the link: http://en.wikipedia.org/wiki/AntiWest
ideology:  Anti-West
ideology ar:  أنتيويست (مترجمه)
ideo ar link:  -1
ideology:  Anti-imperialism
ideology ar:  معاداة الاستعمارية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%8

en.wikipedia.org (parse) History of Hezbollah
en.wikipedia.org (parse) House Foreign Affairs Committee
en.wikipedia.org (parse) Hundred Days' War
en.wikipedia.org (parse) Imperialism
en.wikipedia.org (parse) Inman Report
en.wikipedia.org (parse) Iran
en.wikipedia.org (parse) Iranian Revolution
en.wikipedia.org (imageinfo) File:Mass demonstration in Iran, dat...
en.wikipedia.org (parse) Iraqi embassy bombing in Beirut
en.wikipedia.org (parse) Islamic Jihad Organization
en.wikipedia.org (parse) Islamist
en.wikipedia.org (parse) James Lewis (CIA)
en.wikipedia.org (parse) Janet Lee Stevens
en.wikipedia.org (parse) January 1986 Lebanese Forces coup
en.wikipedia.org (parse) Karantina massacre
en.wikipedia.org (imageinfo) File:Karantina Massacre.jpg
en.wikipedia.org (parse) Kenneth W. Dam
en.wikipedia.org (imageinfo) File:Kenneth Dam.jpg
en.wikipedia.org (parse) Lebanese Civil War
en.wikipedia.org (imageinfo) File:Martyrs Square 1982.jpg
en.wikipedia.org (parse) List of First Ladies of the Un

skipping Wikipedia:Citing sources
skipping Wikipedia:When to cite
skipping Wikipedia:WikiProject Fact and Reference Check
skipping Template:Campaignbox Lebanese Civil War
skipping Template talk:Campaignbox Lebanese Civil War
skipping Help:Maintenance template removal
skipping Category:Articles lacking in-text citations from April 2011
skipping Category:Use mdy dates from March 2012


In [8]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,Hezbollah,"Antisemitism,Khomeinism,Anti-West,Anti-imperia...",-,"Sayyed Hassan Nasrallah,"


In [9]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,حزب الله,"معاداة السامية ,خمينية ,أنتيويست (مترجمه),معاد...",-,حسن نصر الله


In [11]:
datasets_dir = 'datasets_updated/1982-1984/1983_United_States_embassy_bombing_in_Beirut/'

In [11]:
mkdir(datasets_dir)
df_political_party_en.to_csv(os.path.join(datasets_dir, 'political_parties_en.csv'), index=False)
df_political_party_ar.to_csv(os.path.join(datasets_dir, 'political_parties_ar.csv'), index=False, encoding='utf-8-sig')

In [12]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [13]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'
                
                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'
                    
                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    print(name_inserted)
                    if isinstance(name_inserted, list):
                        if len(name_inserted) > 1:
                            print(name_inserted)
                            jsims = []
                            for i, n in enumerate(name_inserted):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name_inserted = [name_inserted[jsims.index(max(jsims))]][0]
                            print('name after: ', name_inserted)
                        else:
                            if name_inserted != []:
                                print('name_inserted before: {}'.format(name_inserted))
                                # name_inserted = [name_inserted]
                                # name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                                name_inserted = name_inserted[0]
                                print('name_inserted after: {}'.format(name_inserted))

                    df_politician_en = df_politician_en.append({
                        'Name': name_inserted,
                        'Political Party': ','.join(party) if isinstance(party, list) else party,
                        'Nationality': ','.join(nationality) if isinstance(nationality, list) else nationality,
                        'Religion': ','.join(religion) if isinstance(religion, list) else religion,
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)
                    
                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)
                    
                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'
                    
                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'
                        
                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'
                    
                    df_politician_ar = df_politician_ar.append({
                        'الاسم': ','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar,
                        'الحزب السياسي': ','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar,
                        'الجنسيه': ','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar,
                        'الدين': ','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar,
                    }, ignore_index=True)
                    
                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)

        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:PikiWiki Israel 4220 Israel Def...
en.wikipedia.org (parse) 1982 Beirut bombing
en.wikipedia.org (parse) 1982 Iranian diplomats kidnapping
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1983 Beirut barracks bombing
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 Beirut barracks bombings
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 Kuwait bombings
en.wikipedia.org (parse) 1983 United States embassy bombing
en.wikipedia.org (imageinfo) File:Beirutembassy.jpg
en.wikipedia.org (parse) 1984 Sohmor massacre
en.wikipedia.org (parse) 1984 United States embassy annex bombing
en.wikipedia.org (imageinfo) File:US Embassy in Beirut, Lebanon (...
en.wikipedia.org (parse) 1984 United States embassy annex bombing...
en.wikipedia.org (imageinfo) File:US Embassy in Beirut, Lebanon (...
en.wikipedia.org (parse) 1985 Beirut car bombing

['Ali Akbar Velayati']
name_inserted before: ['Ali Akbar Velayati']
name_inserted after: Ali Akbar Velayati
-----------------
name:  ['Ali Akbar Velayati']
native_name:  -
nickname:  -
religion:  -
party:  ['Islamic Association of Physicians of Iran', 'Islamic Coalition Party']
nationality:  -
Cannot translate phrase `Ali Akbar Velayati` to arabic
name ar:  -1
START
political party:  Islamic Association of Physicians of Iran
political party ar:  الرابطة الإسلامية لأطباء إيران (مترجمه)
political party ar link:  -1
political party:  Islamic Coalition Party
political party ar:  حزب المؤتلفة الإسلامية 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%85%D8%A4%D8%AA%D9%84%D9%81%D8%A9_%D8%A7%D9%84%D8%A5%D8%B3%D9%84%D8%A7%D9%85%D9%8A%D8%A9
END
name ar:  -1
religion ar:  ['الرابطة الإسلامية لأطباء إيران (مترجمه)', 'حزب المؤتلفة الإسلامية ']
party ar:  ['Islamic Association of Physicians of Iran', 'Islamic Coalition Party']
nationality ar:  -


en.wikipedia.org (parse) Amine Gemayel
en.wikipedia.org (imageinfo) File:C10756-9A (cropped).jpg


['Amine Gemayel']
name_inserted before: ['Amine Gemayel']
name_inserted after: Amine Gemayel
-----------------
name:  ['Amine Gemayel']
native_name:  ['أمين الجميل']
nickname:  -
religion:  -
party:  ['Kataeb Party']
nationality:  ['Lebanese']
name ar:  أمين الجميل 
START
political party:  Kataeb Party
political party ar:  حزب الكتائب اللبنانية 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%83%D8%AA%D8%A7%D8%A6%D8%A8_%D8%A7%D9%84%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
END
START
nationality:  Lebanese
nationality ar:  لبناني (مترجمه)
nationality ar link:  -1
END
name ar:  أمين الجميل 
religion ar:  ['حزب الكتائب اللبنانية ']
party ar:  ['Kataeb Party']
nationality ar:  ['لبناني (مترجمه)']


en.wikipedia.org (parse) Assassination of Bachir Gemayel
en.wikipedia.org (imageinfo) File:Bachir With His Father Pierre (...


...

['Robert Baer']
name_inserted before: ['Robert Baer']
name_inserted after: Robert Baer
-----------------
name:  ['Robert Baer']
native_name:  -
nickname:  -
religion:  -
party:  -
nationality:  ['American']
name ar:  روبرت بير 
START
nationality:  American
nationality ar:  أمريكي (مترجمه)
nationality ar link:  -1
END
name ar:  روبرت بير 
religion ar:  -
party ar:  -
nationality ar:  ['أمريكي (مترجمه)']


en.wikipedia.org (parse) Ronald Reagan
en.wikipedia.org (imageinfo) File:Official Portrait of President ...


-
-----------------
name:  -
native_name:  -
nickname:  -
religion:  -
party:  []
nationality:  -
name ar:  رونالد ريغان 
START
END
name ar:  رونالد ريغان 
religion ar:  []
party ar:  []
nationality ar:  -


en.wikipedia.org (parse) Sabra and Shatila massacre
en.wikipedia.org (imageinfo) File:Bodies of victims of the massac...
en.wikipedia.org (parse) Safra massacre
en.wikipedia.org (parse) Senate Foreign Relations Committee
en.wikipedia.org (imageinfo) File:United States Senate Committee ...
en.wikipedia.org (parse) Siege of Tel al-Zaatar
en.wikipedia.org (imageinfo) File:JeanJacquesKurz-DestructionOfTe...
en.wikipedia.org (parse) Suicide bomber
en.wikipedia.org (parse) Suicide bombing
en.wikipedia.org (parse) Syrian intervention in the Lebanese Civil War
en.wikipedia.org (parse) Terrorist attacks on U.S. diplomatic fac...
en.wikipedia.org (parse) The New York Times
en.wikipedia.org (imageinfo) File:The-New-York-Times-March-26-2018.jpg
en.wikipedia.org (parse) Tribute
en.wikipedia.org (parse) Tyre, Lebanon
en.wikipedia.org (parse) Tyre headquarters bombings
en.wikipedia.org (parse) United States Agency for International D...
en.wikipedia.org (parse) United States Armed Forces
en.wikipedia

-
-----------------
name:  -
native_name:  -
nickname:  -
religion:  -
party:  []
nationality:  ['Israeli']
name ar:  إسحاق شامير 
START
END
START
nationality:  Israeli
nationality ar:  إسرائيلي (مترجمه)
nationality ar link:  -1
END
name ar:  إسحاق شامير 
religion ar:  []
party ar:  []
nationality ar:  ['إسرائيلي (مترجمه)']
skipping Wikipedia:Citing sources
skipping Wikipedia:When to cite
skipping Wikipedia:WikiProject Fact and Reference Check
skipping Template:Campaignbox Lebanese Civil War
skipping Template talk:Campaignbox Lebanese Civil War
skipping Help:Maintenance template removal
skipping Category:Articles lacking in-text citations from April 2011
skipping Category:Use mdy dates from March 2012


In [14]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,Ali Akbar Velayati,"Islamic Association of Physicians of Iran,Isla...",-,-
1,Amine Gemayel,Kataeb Party,Lebanese,-
2,Bachir Gemayel,Kataeb Party,-,-
3,René Moawad,Independent,-,-
4,-,Republican,-,-
5,-,Republican,-,-
6,Claiborne Pell,Democratic,-,-
7,James Foley Lewis,-,-,-
8,Janet Lee Stevens,-,American,-
9,-,-,-,-


In [15]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,-,"الرابطة الإسلامية لأطباء إيران (مترجمه),حزب ال...",-1
1,-,لبناني (مترجمه),حزب الكتائب اللبنانية,أمين الجميل
2,-,-,حزب الكتائب اللبنانية,بشير الجميل
3,-,-,مستقل (توضيح),رينيه معوض
4,-,-,جمهوري (مترجمه),باري جولدواتر
5,-,-,جمهوري (مترجمه),تشارلز هـ بيرسي (مترجمه)
6,-,-,ديمقراطي (مترجمه),كليبورن بيل (مترجمه)
7,-,-,-,جيمس لويس CIA (مترجمه)
8,-,أمريكي (مترجمه),-,جانيت لي ستيفنز (مترجمه)
9,-,-,-,مناحم بيجن


In [16]:
mkdir(datasets_dir)
df_politician_en.to_csv(os.path.join(datasets_dir, 'politicians_en.csv'), index=False)
df_politician_ar.to_csv(os.path.join(datasets_dir, 'politicians_ar.csv'), index=False, encoding='utf-8-sig')

In [6]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [7]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)
                    
                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'
                    
                    name = ','.join(name) if isinstance(name, list) and len(name) >= 1 else name
                    if isEnglish(name):
                        pass
                    else:
                        name = p
                        
                    df_locations_en = df_locations_en.append({
                        'Name': name,
                        'Subdivision type': ','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type,
                        'Subdivision name': ','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name                    
                    }, ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)
                    
                    
                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass
                    
                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'
        

                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = df_locations_ar.append({
                            'الاسم': ','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar,
                            'نوع التقسيم': ','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar,
                            'اسم التقسيم': ','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar,
                        }, ignore_index=True)
                    
                    else:
                        print('skipping {} because name is not found'.format(p))
        
        except LookupError:
            continue
                
            
    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:PikiWiki Israel 4220 Israel Def...
en.wikipedia.org (parse) 1982 Beirut bombing
en.wikipedia.org (parse) 1982 Iranian diplomats kidnapping
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1983 Beirut barracks bombing
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 Beirut barracks bombings
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 Kuwait bombings
en.wikipedia.org (parse) 1983 United States embassy bombing
en.wikipedia.org (imageinfo) File:Beirutembassy.jpg
en.wikipedia.org (parse) 1984 Sohmor massacre
en.wikipedia.org (parse) 1984 United States embassy annex bombing
en.wikipedia.org (imageinfo) File:US Embassy in Beirut, Lebanon (...
en.wikipedia.org (parse) 1984 United States embassy annex bombing...
en.wikipedia.org (imageinfo) File:US Embassy in Beirut, Lebanon (...
en.wikipedia.org (parse) 1985 Beirut car bombing

-----------------
name:  Beirut
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon']
name ar:  بيروت 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
END


en.wikipedia.org (parse) Black Saturday (Lebanon)
en.wikipedia.org (parse) Bombing of Lebanon (June 1981)
en.wikipedia.org (parse) Bureau of Diplomatic Security
en.wikipedia.org (parse) Bus massacre
en.wikipedia.org (parse) CIA
en.wikipedia.org (parse) Car bomb
en.wikipedia.org (parse) Central Intelligence Agency
en.wikipedia.org (parse) Charles H. Percy
en.wikipedia.org (imageinfo) File:Charles Percy.jpg
en.wikipedia.org (parse) Claiborne Pell
en.wikipedia.org (imageinfo) File:Claiborne Pell.jpg
en.wikipedia.org (parse) Crusades
en.wikipedia.org (parse) Dahr al-Wahsh
en.wikipedia.org (parse) Damour massacre
en.wikipedia.org (imageinfo) File:JeanJacquesKurz-DarmourMassacre...
en.wikipedia.org (parse) Diplomatic Security Service
en.wikipedia.org (parse) Ehden massacre
en.wikipedia.org (parse) February 6 Intifada
en.wikipedia.org (parse) GMT
en.wikipedia.org (parse) Geographic coordinate system
en.wikipedia.org (parse) Hezbollah
en.wikipedia.org (parse) History of Hezbollah
en.wikipedia.

-----------------
name:  Iran
subdivision_type:  -
subdivision_name:  -
name ar:  إيران 


en.wikipedia.org (parse) Iranian Revolution
en.wikipedia.org (imageinfo) File:Mass demonstration in Iran, dat...
en.wikipedia.org (parse) Iraqi embassy bombing in Beirut
en.wikipedia.org (parse) Islamic Jihad Organization
en.wikipedia.org (parse) Islamist
en.wikipedia.org (parse) James Lewis (CIA)
en.wikipedia.org (parse) Janet Lee Stevens
en.wikipedia.org (parse) January 1986 Lebanese Forces coup
en.wikipedia.org (parse) Karantina massacre
en.wikipedia.org (imageinfo) File:Karantina Massacre.jpg
en.wikipedia.org (parse) Kenneth W. Dam
en.wikipedia.org (imageinfo) File:Kenneth Dam.jpg
en.wikipedia.org (parse) Lebanese Civil War
en.wikipedia.org (imageinfo) File:Martyrs Square 1982.jpg
en.wikipedia.org (parse) List of First Ladies of the United States
en.wikipedia.org (imageinfo) File:Jill Biden First Lady official ...
en.wikipedia.org (parse) List of Presidents of the United States
en.wikipedia.org (parse) List of embassy attacks
en.wikipedia.org (parse) List of mass car bombings
en.wi

-----------------
name:  Tyre
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon', 'Flag']
name ar:  صور (توضيح) 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
subdivision_name:  Flag
subdivision_name ar:  علم (راية) 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_(%D8%B1%D8%A7%D9%8A%D8%A9)
END


en.wikipedia.org (parse) Tyre headquarters bombings
en.wikipedia.org (parse) United States Agency for International D...
en.wikipedia.org (parse) United States Armed Forces
en.wikipedia.org (parse) War Powers Resolution
en.wikipedia.org (parse) War of Brothers
en.wikipedia.org (parse) War of Liberation (1989–1990)
en.wikipedia.org (parse) War of the Camps
en.wikipedia.org (parse) William R. McIntyre
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Yitzhak Shamir
en.wikipedia.org (imageinfo) File:Yitzhak Shamir 1992 Dan Hadani ...


skipping Wikipedia:Citing sources
skipping Wikipedia:When to cite
skipping Wikipedia:WikiProject Fact and Reference Check
skipping Template:Campaignbox Lebanese Civil War
skipping Template talk:Campaignbox Lebanese Civil War
skipping Help:Maintenance template removal
skipping Category:Articles lacking in-text citations from April 2011
skipping Category:Use mdy dates from March 2012


In [8]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,Beirut,Country,Lebanon
1,Iran,-,-
2,Tyre,Country,"Lebanon,Flag"


In [9]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم
0,لبنان,بلد,بيروت
1,-,-,إيران
2,"لبنان ,علم (راية)",بلد,صور (توضيح)


In [12]:
mkdir(datasets_dir)
df_locations_en.to_csv(os.path.join(datasets_dir, 'locations_en.csv'), index=False)
df_locations_ar.to_csv(os.path.join(datasets_dir, 'locations_ar.csv'), index=False, encoding='utf-8-sig')